# Optimizer(옵티마이저)
모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정.  
1. 옵티마이저 등록  
`optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate`  
2.변화도 명시적으로 0 설정  
`optimizer.zero_grad()`  
3. 예측 손실 역전파  
`loss.bachwards()`  
4. 역전파 단계에서 수집된 변화도로 매개변수 조정   
`optimizer.step()`  
매개변수 = 매개변수 - 학습률 * 그래디언트

In [2]:
# 기본 라이브러리 설치
import torch
import torch.nn as nn
import torch.optim as optim

# Learning Rate Scheduler  
learning rate: gradient 보폭  
Scheduler가 이것을 변경시켜주기도  
`optimizer=torch.optim.SGD(model.parameters(),lr=0.01)` 기본적으로 optimizer 넣어주고

In [ ]:
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

##LambdaLR

`scheduler=optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lambda epoch:0.95**epoch)`  
Parameters
- `optimizer`  
- `lr_lambda` : lr에 곱해질 factor를 정하는 함수

## MultiplicativelLR  
`scheduler=optim.lr_scheduler.MultiplicativeLR(optimizer=optimizer, lr_lambda=lambda epoch:0.95**epoch)`  
parameters
- `optimizer`  
- `lr_lambda`:lr에 곱해질 factor를 정하는

## StepLR  
step size 마다 gamma 비율로 lr 감소시킴  
`scheduler=optim.lr_scheduler.StepLR(optimizer, step_size=10. gamma=0.5)`  
parameters  
- `optimizer`  
- `step_size`: 몇 epoch마다 lr 감소시킬 지
- `gamma`: 얼마의 비율로 감소시킬지

## MultiStepLR
step size가 아니라 learning rate를 감소시킬 epoch을 지정해줌    
  
`scheduler=optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30,80],gamma=0.5)`  
parameters  
- `optimizer`  
- `milestones` 줄일 epoch index의 list
- `gamma`: 어떤 비율로 감소?

## ExponentialLR
learning rate decay가 exponential함수를 따름    
  
`scheduler=optim.lr_scheduler.ExponentialLR(optimizer,gamma=0.5)`  
parameters  
- `optimizer`  
- `gamma`: 어떤 비율?

## CosineAnnealingLR  
learning rate가 cos함수를 따라 eat_min까지 떨어졌다 다시 초기 learningrate까지 올라옴  
  
`scheduler=optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0)`  
parameters  
- `optimizer`
- `T_max`: 최대 iteration 횟수(주기 사이즈)  
- `eta_min`: 최소로 떨어질 수 있는 learing rate default

## RedduceLROnPlateau
성능 향상이 없을 때 learning rate감소. 평가지표를 learning rate step함수의 input으로 넣어줘야 한다.
  
`scheduler=ReduceLROnPlateau(optimizer,'min')`  
`val_loss=validate(...)`  
`scheduler.step(val_loss)` #이렇게 넣어줘야 함  
parameters
- `optimizer`
- `mode`:min/max 감소를 멈출 때, 증가를 멈출 때  
- `factor`: 감소를 시킬 비율  
- `patience` : 향상 안 될 때 몇 epoch 참을 것인가
- `threshold`: 새로운 optimum이 될 수 있는 threshold  
- `threshold_mode: dynamic threshold`: `rel' 'abs'중 하나의 모드로 설정
- `cool_down`: lr이 감소한 후 몇 epoch 동안 동작 쉴지   
- `min_lr`: 최소 lr
- `eps`: 줄인 후 lr차이가 eps보다 작으면 무시

## CosineAnnealingWarmRestarts
cosine annealing 함수 따르면서 Ti epoch 마다 다시 시작  
`scheduler=torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=0.00001)`  
parameters
- `optimizer`
- `T_0` 첫번째 restart를 위해 몇번 iteration
- `T_mult`: restart후에 T_i를 증가시키는 factor  
- `eta_min`: 최소 lr

# 코드로 보기

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from data import AudioDataset, AudioDataLoader
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'data'

In [ ]:
class Model(nn.Module):
  def __init__(self):
    super(Model,self).__init__()
    self.linear=nn.Linear(10,10)
    self.activation=nn.ReLU()
  def forwar(self, x):
    return self.activation(self.linear(x))
#data
tr_dataset=AudioDataset('tr')
data_loader=AudioDataLoader(tr_dataset, batch_size=3, shuffle=1)

#model
model=Model()

#loss
loss=nn.MSELoss()

#optimizer
optimizer=optim.Adam(model.parameters(),lr=1e-3)

#scheduler
scheduler=optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lambda epoch: 0.95**epoch,
                                     last_epoch=-1,
                                     verbose=False)
epochs=100
for epoch in range(epochs):
  for i, (data) in enumerate(data_loader):
    x_data, y_data=data
    optimizer.zero_grad()#1
    loss=loss(y_data, estimated_y)
    loss.backward() #2
    optimizer.step()#3
  scheduler.step() #4


참고  
https://sanghyu.tistory.com/113